In [2]:
import pandas as pd
import sqlite3
import os

database = r'C:\Users\nickf\OneDrive\桌面\Python\python working station\database\nicksdatabase.db'
folder = r'C:\Users\nickf\OneDrive\桌面\Python\python working station\import csv to sqlite database'
csv_container = r'C:\Users\nickf\OneDrive\桌面\Python\python working station\import csv to sqlite database\CSV Converter'

if not os.path.exists(csv_container):
    os.makedirs(csv_container)

conn = sqlite3.connect(database)
cursor = conn.cursor()

file_list=[]
file_names=[]

excel_dic = {}



for root, directory, files in os.walk(folder):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)

            

            #check if targeted worksheet is in the excel file, only extract data that is relevant if yes.
            try:
                xls = pd.ExcelFile(file_path)
                if 'Event Details' in xls.sheet_names:
                    df = pd.read_excel(file_path, sheet_name='Event Details', skiprows=range(4))
                    df = df.iloc[4:]
                    headers = df.columns.tolist()
                    revised_header = [header.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header in headers] 
                    df.columns = revised_header
                    change_to_csv_format = (os.path.join(csv_container, file.replace('.xlsx', '.csv') ))
                    df.to_csv(change_to_csv_format, index=False, encoding='utf_8_sig')
                    file_list.pop(file_list.index(file_path))
                else: 
                    df = pd.read_excel(file_path)
                    headers = df.columns.tolist()
                    revised_header = [header.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header in headers] 
                    df.columns = revised_header
                    change_to_csv_format = (os.path.join(csv_container, file.replace('.xlsx', '.csv') ))
                    df.to_csv(change_to_csv_format, index=False, encoding='utf_8_sig')
         



            except:
                pass

        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            file_list.append(file_path)
            file_name = file.replace(" ","_")
            file_names.append(file_name)
            excel_dic = {file_name:file_list for file_name, file_list in zip(file_names, file_list)}

for file_name, excel_path in excel_dic.items():
    df = pd.read_csv(excel_path)
    sql_revised_header = df.columns
    preparing_header1 = '`'+'` TEXT, `'.join(sql_revised_header)+'` TEXT'
    preparing_header2 = preparing_header1.split(',')
    preparing_header3 =[]
    
    #change data type
    for header in preparing_header2:
        if 'No.' in header or '$' in header or 'Total' in header or 'Booth_Area' in header or 'Net' in header or 'Charges' in header or 'Calculated' in header or 'Branch_Number' in header or 'Net_Order' in header or 'C&E' in header or 'SME' in header or 'product_listing' in header or 'Extended_online' in header or 'booth' in header or 'Package' in header or 'Price' in header or 'Calculated' in header or 'Amount' in header or 'charges' in header or 'eFair_Catalogue' in header or 'Price' in header or 'Unnamed' in header or 'Diff' in header:
            header = header.replace('TEXT', 'INTEGER')
            preparing_header3.append(header)
        elif '8-digit' in header:
            header = header.replace('TEXT', 'TEXT PRIMARY KEY')
            preparing_header3.append(header)
        else:
            preparing_header3.append(header)

            
    cursor.execute (f'CREATE TABLE IF NOT EXISTS `{file_name}` ( {' ,'.join(preparing_header3)});')
    df.to_sql(file_name, conn, if_exists='append', index=False)
    print(f"Successfully imported {file_name} into Database!")


        



conn.close

Successfully imported brand.csv into Database!
Successfully imported t_alibaba_data.csv into Database!
Successfully imported user.csv into Database!
Successfully imported user_brand.csv into Database!


<function Connection.close()>